# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [24]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing, libmpMuelMat, visualization_lines, plot_polarimetry

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
# set the parameters to run the script
directories = [r'E:\test']
calib_directory = r'E:\calib'

# set the parameters to be used for the line visualisation with the parameter parameter_set
parameter_set = 'TheoniPics'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = True

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes with PDDN when available and also without using the PDDN
PDDN_mode = 'no'

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550]

# Processing mode
# 1. 'full': processes everything (azimuth_noise, all plots and visualization) - 1x speed
# 2. 'no_visualization': processes azimuth noise + all plots but no visualization - 1.25x faster
# 3. 'fast': processes only the legacy plots (i.e. depolarization, azimuth, retardance and diattenuation) - 3x faster
processing_mode = 'no_visualization'

# Wether or not time mode (to get the processing times is activated or not)
time_mode = False

In [26]:
times, time_complete = batch_processing.batch_process_master(directories, calib_directory, run_all = run_all, 
                                parameter_set = parameter_set, PDDN = PDDN_mode, 
                                wavelengths = wavelengths, processing_mode = processing_mode,
                                time_mode = time_mode)

processing without PDDN...


  0%|                                                                                            | 0/1 [00:14<?, ?it/s]


NameError: name 'time_viz' is not defined

In [1]:
import pickle
import matplotlib.pyplot as plt
with open(r"C:\Users\romai\Documents\repositories\processingMM\azimuth.pkl", 'rb') as handle:
    [azimuth, sq_size] = pickle.load(handle)

In [4]:

import numpy as np


In [5]:
%%time
import numpy as np
from numba import jit





# Apply the function


CPU times: total: 719 ms
Wall time: 1.71 s
